In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime

In [ ]:
def get_file_info(file_path):
    with open(file_path, 'r') as f:
        data = f.read()
        soup = BeautifulSoup(data, "lxml")

        datum = get_datum(soup)
        print(f"Date: {datum}")

        parteien = list(set([str(partei.text).replace(u'\xa0',u' ') for partei in soup.find_all("p", klasse="AL_Partei")]))
        print(f"Parteien: {len(parteien)}\n{parteien}")

        wahlen = get_wahlen(soup)
        print(f"Durchgeführte Wahlen: {len(wahlen)}")

        entschuldigte = get_entschuldigte(soup)
        print(f"Entschuldigte Abgeordnete: {len(entschuldigte)}\n{entschuldigte}")

        mutterschutz_entschuldigte = get_mutterschutz_entschuldigte(soup)
        print(f"davon im Mutterschutz: {len(mutterschutz_entschuldigte)}\n{mutterschutz_entschuldigte}")

        redner = get_redner(soup)
        print(f"Redner: {len(redner)}\n{redner}")

        redner_fraktionen = get_redner_fraktionen(soup)
        print(f"Redner Fraktionen:\n{redner_fraktionen}")



In [ ]:
def get_df(select_wahlperiode=None):
    datum_list = []
    wahlperiode_list = []
    sitzungsnummer_list = []

    entschuldigte_list = []
    entschuldigte_fraktionen_list = []
    davon_mutterschutz_list = []

    wahlen_list = []

    redner_list = []

    redner_fraktionen_list = []

    for folder in os.listdir(DOWNLOAD_FOLDER_NAME):
        print(folder)
        if select_wahlperiode is not None and str(select_wahlperiode) not in folder:
            print(f"Skipping {folder}")
            continue
        for file in os.listdir(Path(DOWNLOAD_FOLDER_NAME, folder)):
            if file.endswith(".xml"):
                print(f"Auswertung von {file}")
                with open(Path(DOWNLOAD_FOLDER_NAME, folder, file), 'r') as f:
                    soup = BeautifulSoup(f.read(), "lxml")

                    datum = datetime.strptime(get_datum(soup), "%d.%m.%Y")
                    # print(f"Datum: {datum}")
                    datum_list.append(datum)

                    wahlperiode = get_wahlperiode(soup)
                    # print(f"Wahlperiode: {wahlperiode}")
                    wahlperiode_list.append(wahlperiode)

                    sitzungsnummer = get_sitzungsnummer(soup)
                    # print(f"Sitzungsnummer: {sitzungsnummer}")
                    sitzungsnummer_list.append(sitzungsnummer)

                    entschuldigte = get_entschuldigte(soup)
                    # print(f"Entschuldigte Abgeordnete: {len(entschuldigte)}\n{entschuldigte}")
                    entschuldigte_list.append(len(entschuldigte))

                    mutterschutz_entschuldigte = get_mutterschutz_entschuldigte(soup)
                    # print(f"davon im Mutterschutz: {len(mutterschutz_entschuldigte)}\n{mutterschutz_entschuldigte}")
                    davon_mutterschutz_list.append(len(mutterschutz_entschuldigte))

                    entschuldigte_fraktionen = get_entschuldigte_fraktionen(soup)
                    # print(f"Entschuldigte Fraktionen:\n{entschuldigte_fraktionen}")
                    entschuldigte_fraktionen_list.append(entschuldigte_fraktionen)

                    anzahl_wahlen = len(get_wahlen(soup))
                    # print(f"Durchgeführte Wahlen: {anzahl_wahlen}")
                    wahlen_list.append(anzahl_wahlen)

                    anzahl_redner = len(get_redner(soup))
                    # print(f"Redner: {anzahl_redner}")
                    redner_list.append(anzahl_redner)

                    redner_fraktionen = get_redner_fraktionen(soup)
                    # print(f"Redner Fraktionen:\n{redner_fraktionen}")
                    redner_fraktionen_list.append(redner_fraktionen)
                    
    
    
    df_data = {
        ("metadata", "datum"):datum_list,
        ("metadata", "wahlperiode"):wahlperiode_list,
        ("metadata", "sitzungsnummer"):sitzungsnummer_list,
        ("entschuldigungen", "gesamt"):entschuldigte_list,
        ("entschuldigungen", "mutterschutz"):davon_mutterschutz_list,
        ("abstimmungen", "wahlen"):wahlen_list,
        ("reden", "gesamt"):redner_list
    }

    for i, fraktion in enumerate(FRAKTIONEN):
        df_data[("reden", fraktion)] = [redner_fraktionen[fraktion] for redner_fraktionen in redner_fraktionen_list]
        df_data[("entschuldigungen", fraktion)] = [entschuldigte_fraktionen[fraktion] for entschuldigte_fraktionen in entschuldigte_fraktionen_list]
    
    df = pd.DataFrame(df_data, index=pd.to_datetime(datum_list))
    
    df.sort_index(inplace=True)
    return df

In [ ]:
def get_wahlen(soup):
    wahlen_divs = soup.select('anlagen-text[anlagen-typ="Ergebnis und Namensverzeichnis"]')
    wahlen = []
    for wahl_div in wahlen_divs:
        if wahl_div.find("p", klasse="T_fett") is not None:
            wahltitel = wahl_div.find("p", klasse="T_fett").text.replace(u'\xa0',u' ').replace("der Mitglieder des Deutschen Bundestages, die an der ","").replace("teilgenommen haben","")
        else:
            wahltitel = wahl_div.find("p", klasse="Anlage_3").text.replace(u'\xa0',u' ').replace("der Mitglieder des Deutschen Bundestages, die an der ","").replace("teilgenommen haben","")
        
            
        if wahl_div.table is not None and wahl_div.table.caption is not None:
            abgegebene_stimmen = wahl_div.table.caption.text.replace(u'\xa0',u' ').replace("Abgegebene Stimmkarten: ","").replace(" Ergebnis","")
        else:
            abgegebene_stimmen = wahl_div.find(lambda tag:tag.name=="p" and "Abgegebene Stimmkarten" in tag.text.replace(u'\xa0',u' ')).text.replace(u'\xa0',u' ').replace("Abgegebene Stimmkarten: ","").replace(" Ergebnis","")
        
        if " Für die Wahl sind mindestens" in abgegebene_stimmen:
            abgegebene_stimmen = abgegebene_stimmen.split(" Für die Wahl sind mindestens")[0]
        abgegebene_stimmen = int(abgegebene_stimmen)
        print(f"Wahl: {wahltitel}, Abgegebene Stimmen: {abgegebene_stimmen}")
        wahlen.append({"titel":wahltitel, "stimmen":abgegebene_stimmen})
    
    return wahlen

In [ ]:
def get_wahlperiode(soup):
    wahlperiode = int(soup.find("wahlperiode").text)
    return wahlperiode

In [ ]:
def get_sitzungsnummer(soup):
    sitzungsnummer = int(soup.find("sitzungsnr").text)
    return sitzungsnummer

In [ ]:
def get_datum(soup):
    date = soup.find_all("datum")[0]["date"]
    return date

In [ ]:
def get_redner(soup):
    rednerliste_div = soup.find("rednerliste")
    redner = [
        get_normale_namen(redner.find("titel").text + " " + redner.find("vorname").text + " " +  redner.find("nachname").text)
        if redner.find("titel") is not None 
        else get_normale_namen(redner.find("vorname").text + " " +  redner.find("nachname").text)
        for redner in rednerliste_div.find_all("redner")]
    redner = list(set(redner))
    return redner

In [ ]:
def get_redner_fraktionen(soup):
    rednerliste_div = soup.find("rednerliste")
    redner_fraktionen = [
        redner.find("fraktion").text.replace(u'\xa0',u' ')
        if redner.find("fraktion") is not None
        else "unbekannt"
        for redner in rednerliste_div.find_all("redner")
    ]
    vorkommen = pd.value_counts(redner_fraktionen)
    for fraktion in FRAKTIONEN:
        if fraktion not in vorkommen.index:
            vorkommen[fraktion] = 0
    for fraktion in vorkommen.index:
        if fraktion not in FRAKTIONEN:
            vorkommen["unbekannt"] += vorkommen[fraktion]
            vorkommen.drop(fraktion, inplace=True)
    vorkommen.sort_index(inplace=True)
    return vorkommen.to_dict()

In [ ]:
def get_entschuldigte_fraktionen(soup):
    entschuldigte_div = soup.select_one('anlagen-text[anlagen-typ="Entschuldigte Abgeordnete"]')
    if entschuldigte_div is None:
        entschuldigte_div = soup.select_one('anlagen-text[anlagen-typ="Liste der entschuldigten Abgeordneten"]')
        if entschuldigte_div is None:
            print("Keine entschuldigten Abgeordneten gefunden")
            return dict(zip(FRAKTIONEN, [0]*len(FRAKTIONEN)))
    if entschuldigte_div.tbody is None:
        print("Keine entschuldigten Abgeordneten gefunden")
        return dict(zip(FRAKTIONEN, [0]*len(FRAKTIONEN)))
    entschuldigte_fraktionen = [
        tr.find_all("td")[1].text.replace(u'\xa0',u' ').replace("BÜNDNIS 90/ DIE GRÜNEN","BÜNDNIS 90/DIE GRÜNEN").replace("fraktionslos","Fraktionslos")
        for tr in entschuldigte_div.tbody.find_all("tr")
        if tr.td is not None
    ]
    vorkommen = pd.value_counts(entschuldigte_fraktionen)
    for fraktion in FRAKTIONEN:
        if fraktion not in vorkommen.index:
            vorkommen[fraktion] = 0
    for fraktion in vorkommen.index:
        if fraktion not in FRAKTIONEN:
            vorkommen["unbekannt"] += vorkommen[fraktion]
            vorkommen.drop(fraktion, inplace=True)
    vorkommen.sort_index(inplace=True)
    return vorkommen.to_dict()

In [ ]:
def get_entschuldigte(soup):
    entschuldigte_div = soup.select_one('anlagen-text[anlagen-typ="Entschuldigte Abgeordnete"]')
    if entschuldigte_div is None:
        entschuldigte_div = soup.select_one('anlagen-text[anlagen-typ="Liste der entschuldigten Abgeordneten"]')
        if entschuldigte_div is None:
            print("Keine entschuldigten Abgeordneten gefunden")
            return []
    if entschuldigte_div.tbody is None:
        print("Keine entschuldigten Abgeordneten gefunden")
        return []
    entschuldigte = [
        get_normale_namen(tr.td.text) for tr in entschuldigte_div.tbody.find_all("tr")
        if tr.td is not None
    ]
    return entschuldigte

In [ ]:
def get_mutterschutz_entschuldigte(soup):
    entschuldigte_div = soup.select_one('anlagen-text[anlagen-typ="Entschuldigte Abgeordnete"]')
    if entschuldigte_div is None:
        entschuldigte_div = soup.select_one('anlagen-text[anlagen-typ="Liste der entschuldigten Abgeordneten"]')
        if entschuldigte_div is None:
            print("Keine entschuldigten Abgeordneten gefunden")
            return []
    if entschuldigte_div.tbody is None:
        print("Keine entschuldigten Abgeordneten gefunden")
        return []
    mutterschutz_entschuldigte = [
        get_normale_namen(tr.td.text) for tr in entschuldigte_div.tbody.find_all("tr") 
        if tr.td is not None
        and ("*" in tr.td.text or "(aufgrund gesetzlichen Mutterschutzes)" in tr.td.text)
    ]
    return mutterschutz_entschuldigte

In [ ]:
def get_normale_namen(name):
    name = name.replace("*","").replace("(aufgrund gesetzlichen Mutterschutzes)","").replace(u'\xa0',u' ')
    if "," in name:
        split_name = name.split(",")
        name = split_name[1] + " " + split_name[0]
    name = name.replace("  "," ")
    if name[0] == " ":
        name = name[1:]
    if name[-1] == " ":
        name = name[:-1]
    return name